---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
len(states)

57

## Question 1

In [4]:
UT=pd.read_table('university_towns.txt', header = None)
UT.head(10)

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


In [5]:
UT_State = UT.copy()
UT_State = UT_State[UT_State[0].str.contains(r"\[edit\]")]


In [6]:
UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
len(UT_State_List)

50

In [7]:
UT[0]=UT[0].str.replace(r"\(.*\)","").str.replace(r"\[.*\]","").str.strip()
print("Type ",type(UT))
print("Length ",len(UT))
UT.head(10)

Type  <class 'pandas.core.frame.DataFrame'>
Length  567


,0
0,Alabama
1,Auburn
2,Florence
3,Jacksonville
4,Livingston
5,Montevallo
6,Troy
7,Tuscaloosa
8,Tuskegee
9,Alaska


In [8]:
UT.loc[0,0]

'Alabama'

In [9]:
UT.loc[0,0] in UT_State_List

True

In [10]:
data = []

for index, row in UT.iterrows():
    if(UT.loc[index,0] in UT_State_List):
        state = UT.loc[index,0]
        UT_State_List.remove(state)
    else:
        town = UT.loc[index,0]
        data.append([state,town])

In [11]:
unitowns = pd.DataFrame(data,columns=["State","RegionName"])
unitowns.shape

(517, 2)

In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    UT=pd.read_table('university_towns.txt', header = None)
    
    UT_State = UT[UT[0].str.contains(r"\[edit\]")]
    UT_State_List = UT_State[0].str.replace(r"\[.*\]","").str.strip().tolist()
    
    UT[0]=UT[0].str.replace(r"\(.*","").str.replace(r"\[.*\]","").str.rstrip()
    
    data = []

    for index, row in UT.iterrows():
        if(UT.loc[index,0] in UT_State_List):
            state = UT.loc[index,0]
            UT_State_List.remove(state)
        else:
            town = UT.loc[index,0]
            data.append([state,town])

    unitowns = pd.DataFrame(data,columns=["State","RegionName"])
    
    return unitowns

In [13]:
df = get_list_of_university_towns()

In [14]:
cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')

Shape test:  Passed


In [15]:
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

Index test:  Passed


In [16]:
print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')

Column test:  Passed


In [17]:
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')

\n test:  Passed


In [18]:
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
              '\s+$'))
      else 'Passed')

Trailing whitespace test: Passed


In [19]:
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
              '\('))
      else 'Passed')

"(" test: Passed


## Question 2

In [20]:
GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
GDP.head(10)

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,2000q1,10031.0,12359.1,NaN
1,NaN,NaN,NaN,NaN,2000q2,10278.3,12592.5,NaN
2,NaN,NaN,NaN,NaN,2000q3,10357.4,12607.7,NaN
3,NaN,NaN,NaN,NaN,2000q4,10472.3,12679.3,NaN
4,NaN,NaN,NaN,NaN,2001q1,10508.1,12643.3,NaN
5,NaN,NaN,NaN,NaN,2001q2,10638.4,12710.3,NaN
6,NaN,NaN,NaN,NaN,2001q3,10639.5,12670.1,NaN
7,NaN,NaN,NaN,NaN,2001q4,10701.3,12705.3,NaN
8,NaN,NaN,NaN,NaN,2002q1,10834.4,12822.3,NaN
9,NaN,NaN,NaN,NaN,2002q2,10934.8,12893.0,NaN


In [21]:
GDP=GDP[[4,5,6]]
print(GDP.shape)
GDP.head(10)

(66, 3)


,4,5,6
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3
5,2001q2,10638.4,12710.3
6,2001q3,10639.5,12670.1
7,2001q4,10701.3,12705.3
8,2002q1,10834.4,12822.3
9,2002q2,10934.8,12893.0


In [22]:
recession_start = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
        recession_start.append(GDP.iloc[i][4])

In [23]:
recession_start

['2008q3', '2008q4']

In [24]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows=220,header = None)
    GDP=GDP[[4,5,6]]
    
    recession_start = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] > GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] > GDP.iloc[i+2][5]):
            recession_start.append(GDP.iloc[i][4])
    
    return recession_start[0]

In [25]:
recession_start = get_recession_start()
recession_start

'2008q3'

## Problem 3

In [93]:
recession_end = []
for i in range(len(GDP) - 2):
    if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
        recession_end.append(GDP.iloc[i+2][4])

In [94]:
recession_end = [i for i in recession_end if i > recession_start]

In [95]:
recession_end[0]

'2008q3'

In [29]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_end = []
    for i in range(len(GDP) - 2):
        if(GDP.iloc[i][5] < GDP.iloc[i+1][5]) & (GDP.iloc[i+1][5] < GDP.iloc[i+2][5]):
            recession_end.append(GDP.iloc[i+2][4])
    
    recession_end = [i for i in recession_end if i > recession_start]
    
    return recession_end[0]

In [30]:
recession_end = get_recession_end()
recession_end

'2009q4'

## Problem 4

### Approach 1

In [31]:
GDP[GDP[4] == recession_start].index[0]

34

In [32]:
GDP[GDP[4] == recession_end].index[0]

39

In [33]:
GDP.iloc[34:39]

,4,5,6
34,2008q3,14843.0,14891.6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [34]:
recession = GDP.iloc[GDP[GDP[4] == recession_start].index[0]:GDP[GDP[4] == recession_end].index[0]]
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

### Approach 2

In [35]:
recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
recession

,4,5,6
35,2008q4,14549.9,14577.0
36,2009q1,14383.9,14375.0
37,2009q2,14340.4,14355.6
38,2009q3,14384.1,14402.5


In [36]:
recession[recession[5]==recession[5].min()].iloc[0][4]

'2009q2'

In [37]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession = GDP[(GDP[4] > recession_start) & (GDP[4] < recession_end)]
    
    return recession[recession[5]==recession[5].min()].iloc[0][4]

In [38]:
get_recession_bottom()

'2009q2'

## Problem 5

In [39]:
Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
Zillow.head(10)

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,18959,Las Vegas,NV,Las Vegas,Clark,6,121600.0,120900.0,120400.0,120300.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,54296,San Diego,CA,San Diego,San Diego,7,161100.0,160700.0,160400.0,160100.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,38128,Dallas,TX,Dallas-Fort Worth,Dallas,8,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,33839,San Jose,CA,San Jose,Santa Clara,9,224500.0,224900.0,225400.0,226100.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,25290,Jacksonville,FL,Jacksonville,Duval,10,77500.0,77200.0,76800.0,76600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [40]:
print(Zillow.columns)

Index(['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '1996-04', '1996-05', '1996-06', '1996-07',
       ...
       '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
       '2016-05', '2016-06', '2016-07', '2016-08'],
      dtype='object', length=251)


In [41]:
Zillow[['1996-04', '1996-05', '1996-06']].mean(axis=1).head(10)

0              NaN
1    154666.666667
2    109466.666667
3     49833.333333
4     87700.000000
5    120966.666667
6    160733.333333
7              NaN
8    224933.333333
9     77166.666667
dtype: float64

In [42]:
Zillow[['1996-04', '1996-05', '1996-06']].mean(axis=0)

1996-04    120373.809229
1996-05    120454.175327
1996-06    120338.756388
dtype: float64

In [43]:
'1996-04'.split('-')

['1996', '04']

In [44]:
'04' >= '01'

True

In [45]:
List = '1996-01'.split('-')

In [46]:
Year = List[0]
Month = List[1]
print("Year ",Year)
print("Month ",Month)

Year  1996
Month  01


In [47]:
def month_to_quarter(String):
    List = String.split('-')
    Year = List[0]
    Month = List[1]
    Quarter=''
    if (Month >= '01') & (Month <= '03'):
        Quarter='q1'
    elif (Month >= '04') & (Month <= '06'):
        Quarter='q2'
    elif (Month >= '07') & (Month <= '09'):
        Quarter='q3'
    elif (Month >= '10') & (Month <= '12'):
        Quarter='q4'
    return Year+Quarter

In [48]:
print(month_to_quarter('1996-01'))
print(month_to_quarter('1996-02'))
print(month_to_quarter('1996-03'))
print(month_to_quarter('1996-04'))
print(month_to_quarter('1996-05'))
print(month_to_quarter('1996-06'))
print(month_to_quarter('1996-07'))
print(month_to_quarter('1996-08'))
print(month_to_quarter('1996-09'))
print(month_to_quarter('1996-10'))
print(month_to_quarter('1996-11'))
print(month_to_quarter('1996-12'))

1996q1
1996q1
1996q1
1996q2
1996q2
1996q2
1996q3
1996q3
1996q3
1996q4
1996q4
1996q4


In [49]:
Quarter_List = []
Months = ['1996-04','1996-05','1996-06','1996-07','1996-08','1996-09','1996-10','1996-11','1996-12','1997-01','1997-02']
for month in Months:
    temp = month_to_quarter(month)
    if(temp not in Quarter_List):
        Quarter_List.append(temp)
Quarter_List

['1996q2', '1996q3', '1996q4', '1997q1']

In [50]:
Quarter_List = []
for month in Months:
    temp = month_to_quarter(month)
    print(Zillow.iloc[1][month])
    if(temp not in Quarter_List):
        Quarter_List.append(temp)
Quarter_List

155000.0
154600.0
154400.0
154200.0
154100.0
154300.0
154300.0
154200.0
154800.0
155900.0
157000.0


['1996q2', '1996q3', '1996q4', '1997q1']

In [51]:
Quarter_List = []
for month in Months:
    temp = month_to_quarter(month)
    if(temp not in Quarter_List):
        print(temp)
        Quarter_List.append(temp)

1996q2
1996q3
1996q4
1997q1


In [52]:
Quarter_List = []
One_Quarter = []
for month in Months:
    One_Quarter.append(month)
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter)
        print(One_Quarter)
        One_Quarter = []
Quarter_List

['1996-04']
['1996-05', '1996-06', '1996-07']
['1996-08', '1996-09', '1996-10']
['1996-11', '1996-12', '1997-01']


['1996q2', '1996q3', '1996q4', '1997q1']

In [53]:
Quarter_List = []
One_Quarter = []
Month_List = []
for month in Months:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            print(One_Quarter)
            Month_List.append(One_Quarter)
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        if(Months[-1] in One_Quarter):
            print(One_Quarter)
            Month_List.append(One_Quarter)

['1996-04', '1996-05', '1996-06']
['1996-07', '1996-08', '1996-09']
['1996-10', '1996-11', '1996-12']
['1997-01', '1997-02']


In [54]:
print(Month_List)
print(Quarter_List)

[['1996-04', '1996-05', '1996-06'], ['1996-07', '1996-08', '1996-09'], ['1996-10', '1996-11', '1996-12'], ['1997-01', '1997-02']]
['1996q2', '1996q3', '1996q4', '1997q1']


In [55]:
dictionary = dict(zip(Quarter_List, Month_List))
dictionary

{'1996q2': ['1996-04', '1996-05', '1996-06'],
 '1996q3': ['1996-07', '1996-08', '1996-09'],
 '1996q4': ['1996-10', '1996-11', '1996-12'],
 '1997q1': ['1997-01', '1997-02']}

In [56]:
Quarter_List = []
One_Quarter = []
for month in Months:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        

        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        if(Months[-1] in One_Quarter):
            print(One_Quarter) 

['1997-01', '1997-02']


In [57]:
Quarter_List = []
One_Quarter = []
Quarter_dict = {}
for month in Months:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            Quarter_dict[currnt_quarter] =One_Quarter
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        Quarter_dict[currnt_quarter] =One_Quarter
        if(Months[-1] in One_Quarter):
            Quarter_dict[currnt_quarter] =One_Quarter
Quarter_dict

{'1996q2': ['1996-04', '1996-05', '1996-06'],
 '1996q3': ['1996-07', '1996-08', '1996-09'],
 '1996q4': ['1996-10', '1996-11', '1996-12'],
 '1997q1': ['1997-01', '1997-02']}

In [58]:
Quarter_List = []
One_Quarter = []
Quarter_dict = {}
for month in Months:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            Quarter_dict[currnt_quarter] =One_Quarter
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        Quarter_dict[currnt_quarter] =One_Quarter
        if(Months[-1] in One_Quarter):
            Quarter_dict[currnt_quarter] =One_Quarter
Quarter_dict

{'1996q2': ['1996-04', '1996-05', '1996-06'],
 '1996q3': ['1996-07', '1996-08', '1996-09'],
 '1996q4': ['1996-10', '1996-11', '1996-12'],
 '1997q1': ['1997-01', '1997-02']}

In [59]:
DIC_Test={}
DIC_Test['2']=[1]
DIC_Test

{'2': [1]}

In [60]:
DIC_Test['2']=[1,2,3]
DIC_Test

{'2': [1, 2, 3]}

In [61]:
Quarter_List = []
One_Quarter = []
Quarter_dict = {}
for month in Months:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            Quarter_dict[currnt_quarter] =One_Quarter
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        Quarter_dict[currnt_quarter] =One_Quarter
Quarter_dict

{'1996q2': ['1996-04', '1996-05', '1996-06'],
 '1996q3': ['1996-07', '1996-08', '1996-09'],
 '1996q4': ['1996-10', '1996-11', '1996-12'],
 '1997q1': ['1997-01', '1997-02']}

In [62]:
for k, v in Quarter_dict.items():
    print(k,v)

1997q1 ['1997-01', '1997-02']
1996q3 ['1996-07', '1996-08', '1996-09']
1996q4 ['1996-10', '1996-11', '1996-12']
1996q2 ['1996-04', '1996-05', '1996-06']


In [63]:
Zillow_column_names = Zillow.columns.tolist()
print(len(Zillow_column_names))
Zillow_column_names[0]

251


'RegionID'

In [64]:
Zillow_column_names[0][0].isdigit()

False

In [65]:
Zillow_column_month = [x for x in Zillow_column_names if x[0].isdigit()]
print(len(Zillow_column_month))
Zillow_column_month[0]

245


'1996-04'

In [66]:
Quarter_List = []
One_Quarter = []
Quarter_dict = {}
for month in Zillow_column_month:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            Quarter_dict[currnt_quarter] =One_Quarter
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        Quarter_dict[currnt_quarter] =One_Quarter
Quarter_dict

{'1996q2': ['1996-04', '1996-05', '1996-06'],
 '1996q3': ['1996-07', '1996-08', '1996-09'],
 '1996q4': ['1996-10', '1996-11', '1996-12'],
 '1997q1': ['1997-01', '1997-02', '1997-03'],
 '1997q2': ['1997-04', '1997-05', '1997-06'],
 '1997q3': ['1997-07', '1997-08', '1997-09'],
 '1997q4': ['1997-10', '1997-11', '1997-12'],
 '1998q1': ['1998-01', '1998-02', '1998-03'],
 '1998q2': ['1998-04', '1998-05', '1998-06'],
 '1998q3': ['1998-07', '1998-08', '1998-09'],
 '1998q4': ['1998-10', '1998-11', '1998-12'],
 '1999q1': ['1999-01', '1999-02', '1999-03'],
 '1999q2': ['1999-04', '1999-05', '1999-06'],
 '1999q3': ['1999-07', '1999-08', '1999-09'],
 '1999q4': ['1999-10', '1999-11', '1999-12'],
 '2000q1': ['2000-01', '2000-02', '2000-03'],
 '2000q2': ['2000-04', '2000-05', '2000-06'],
 '2000q3': ['2000-07', '2000-08', '2000-09'],
 '2000q4': ['2000-10', '2000-11', '2000-12'],
 '2001q1': ['2001-01', '2001-02', '2001-03'],
 '2001q2': ['2001-04', '2001-05', '2001-06'],
 '2001q3': ['2001-07', '2001-08', 

In [67]:
{k: v for k, v in Quarter_dict.items() if k >= '2001q1'}

{'2001q1': ['2001-01', '2001-02', '2001-03'],
 '2001q2': ['2001-04', '2001-05', '2001-06'],
 '2001q3': ['2001-07', '2001-08', '2001-09'],
 '2001q4': ['2001-10', '2001-11', '2001-12'],
 '2002q1': ['2002-01', '2002-02', '2002-03'],
 '2002q2': ['2002-04', '2002-05', '2002-06'],
 '2002q3': ['2002-07', '2002-08', '2002-09'],
 '2002q4': ['2002-10', '2002-11', '2002-12'],
 '2003q1': ['2003-01', '2003-02', '2003-03'],
 '2003q2': ['2003-04', '2003-05', '2003-06'],
 '2003q3': ['2003-07', '2003-08', '2003-09'],
 '2003q4': ['2003-10', '2003-11', '2003-12'],
 '2004q1': ['2004-01', '2004-02', '2004-03'],
 '2004q2': ['2004-04', '2004-05', '2004-06'],
 '2004q3': ['2004-07', '2004-08', '2004-09'],
 '2004q4': ['2004-10', '2004-11', '2004-12'],
 '2005q1': ['2005-01', '2005-02', '2005-03'],
 '2005q2': ['2005-04', '2005-05', '2005-06'],
 '2005q3': ['2005-07', '2005-08', '2005-09'],
 '2005q4': ['2005-10', '2005-11', '2005-12'],
 '2006q1': ['2006-01', '2006-02', '2006-03'],
 '2006q2': ['2006-04', '2006-05', 

### Approach 1

In [68]:
Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
Zillow.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
Zillow['State'] = Zillow['State'].map(states)
Zillow.set_index(['State','RegionName'],inplace=True)
col = list(Zillow.columns)
col = col[0:45]
Zillow.drop(col,axis=1,inplace=1)

In [69]:
Zillow.head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [70]:
def month_to_quarter(String):
    List = String.split('-')
    Year = List[0]
    Month = List[1]
    Quarter=''
    if (Month >= '01') & (Month <= '03'):
        Quarter='q1'
    elif (Month >= '04') & (Month <= '06'):
        Quarter='q2'
    elif (Month >= '07') & (Month <= '09'):
        Quarter='q3'
    elif (Month >= '10') & (Month <= '12'):
        Quarter='q4'
    return Year+Quarter

In [71]:
Zillow_column_names = Zillow.columns.tolist()
Zillow_column_month = [x for x in Zillow_column_names if x[0].isdigit()]
Quarter_List = []
One_Quarter = []
Quarter_dict = {}
for month in Zillow_column_month:
    currnt_quarter = month_to_quarter(month)
    if(currnt_quarter not in Quarter_List):
        Quarter_List.append(currnt_quarter) 
        if(len(One_Quarter)!=0):
            Quarter_dict[currnt_quarter] =One_Quarter
        One_Quarter = []
        One_Quarter.append(month)
    else:
        One_Quarter.append(month)
        Quarter_dict[currnt_quarter] =One_Quarter
Quarter_dict

{'2000q1': ['2000-01', '2000-02', '2000-03'],
 '2000q2': ['2000-04', '2000-05', '2000-06'],
 '2000q3': ['2000-07', '2000-08', '2000-09'],
 '2000q4': ['2000-10', '2000-11', '2000-12'],
 '2001q1': ['2001-01', '2001-02', '2001-03'],
 '2001q2': ['2001-04', '2001-05', '2001-06'],
 '2001q3': ['2001-07', '2001-08', '2001-09'],
 '2001q4': ['2001-10', '2001-11', '2001-12'],
 '2002q1': ['2002-01', '2002-02', '2002-03'],
 '2002q2': ['2002-04', '2002-05', '2002-06'],
 '2002q3': ['2002-07', '2002-08', '2002-09'],
 '2002q4': ['2002-10', '2002-11', '2002-12'],
 '2003q1': ['2003-01', '2003-02', '2003-03'],
 '2003q2': ['2003-04', '2003-05', '2003-06'],
 '2003q3': ['2003-07', '2003-08', '2003-09'],
 '2003q4': ['2003-10', '2003-11', '2003-12'],
 '2004q1': ['2004-01', '2004-02', '2004-03'],
 '2004q2': ['2004-04', '2004-05', '2004-06'],
 '2004q3': ['2004-07', '2004-08', '2004-09'],
 '2004q4': ['2004-10', '2004-11', '2004-12'],
 '2005q1': ['2005-01', '2005-02', '2005-03'],
 '2005q2': ['2005-04', '2005-05', 

In [72]:
for k, v in Quarter_dict.items():
    Zillow[k] = Zillow[v].mean(axis=1)

In [73]:
Quarter_Names = list(Quarter_dict.keys())
Quarter_Names.sort()

In [74]:
Zillow = Zillow[Quarter_Names]

In [75]:
Zillow

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


### Approach 2

In [76]:
Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
Zillow.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
Zillow.head(10)

,RegionName,State,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,Los Angeles,CA,155000.0,154600.0,154400.0,154200.0,154100.0,154300.0,154300.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,Chicago,IL,109700.0,109400.0,109300.0,109300.0,109100.0,109000.0,109000.0,109600.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,Philadelphia,PA,50000.0,49900.0,49600.0,49400.0,49400.0,49300.0,49300.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,Phoenix,AZ,87200.0,87700.0,88200.0,88400.0,88500.0,88900.0,89400.0,89700.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,Las Vegas,NV,121600.0,120900.0,120400.0,120300.0,120200.0,120300.0,120500.0,120900.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,San Diego,CA,161100.0,160700.0,160400.0,160100.0,159600.0,159100.0,158700.0,158500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,Dallas,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,San Jose,CA,224500.0,224900.0,225400.0,226100.0,227100.0,228100.0,229300.0,230900.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,Jacksonville,FL,77500.0,77200.0,76800.0,76600.0,76300.0,76000.0,75900.0,76000.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [77]:
Zillow['State'] = Zillow['State'].map(states)
Zillow.set_index(['State','RegionName'],inplace=True)
Zillow.head(10)

,,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,155000.0,154600.0,154400.0,154200.0,154100.0,154300.0,154300.0,154200.0,154800.0,155900.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,109700.0,109400.0,109300.0,109300.0,109100.0,109000.0,109000.0,109600.0,110200.0,110800.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,50000.0,49900.0,49600.0,49400.0,49400.0,49300.0,49300.0,49400.0,49700.0,49600.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,87200.0,87700.0,88200.0,88400.0,88500.0,88900.0,89400.0,89700.0,90100.0,90700.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
Nevada,Las Vegas,121600.0,120900.0,120400.0,120300.0,120200.0,120300.0,120500.0,120900.0,121400.0,122100.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
California,San Diego,161100.0,160700.0,160400.0,160100.0,159600.0,159100.0,158700.0,158500.0,158700.0,159800.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
Texas,Dallas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
California,San Jose,224500.0,224900.0,225400.0,226100.0,227100.0,228100.0,229300.0,230900.0,233400.0,236500.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700


In [78]:
col = list(Zillow.columns)
col = col[0:45]
Zillow.drop(col,axis=1,inplace=1)
Zillow.head(10)

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
Nevada,Las Vegas,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
California,San Diego,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
Texas,Dallas,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
California,San Jose,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700


In [79]:
qs = [list(Zillow.columns)[x:x+3] for x in range(0, len(list(Zillow.columns)), 3)]
qs

[['2000-01', '2000-02', '2000-03'],
 ['2000-04', '2000-05', '2000-06'],
 ['2000-07', '2000-08', '2000-09'],
 ['2000-10', '2000-11', '2000-12'],
 ['2001-01', '2001-02', '2001-03'],
 ['2001-04', '2001-05', '2001-06'],
 ['2001-07', '2001-08', '2001-09'],
 ['2001-10', '2001-11', '2001-12'],
 ['2002-01', '2002-02', '2002-03'],
 ['2002-04', '2002-05', '2002-06'],
 ['2002-07', '2002-08', '2002-09'],
 ['2002-10', '2002-11', '2002-12'],
 ['2003-01', '2003-02', '2003-03'],
 ['2003-04', '2003-05', '2003-06'],
 ['2003-07', '2003-08', '2003-09'],
 ['2003-10', '2003-11', '2003-12'],
 ['2004-01', '2004-02', '2004-03'],
 ['2004-04', '2004-05', '2004-06'],
 ['2004-07', '2004-08', '2004-09'],
 ['2004-10', '2004-11', '2004-12'],
 ['2005-01', '2005-02', '2005-03'],
 ['2005-04', '2005-05', '2005-06'],
 ['2005-07', '2005-08', '2005-09'],
 ['2005-10', '2005-11', '2005-12'],
 ['2006-01', '2006-02', '2006-03'],
 ['2006-04', '2006-05', '2006-06'],
 ['2006-07', '2006-08', '2006-09'],
 ['2006-10', '2006-11', '200

In [80]:
def new_col_names():
    #generating the new coloumns names 
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]

In [81]:
column_names = new_col_names()
for col,q in zip(column_names,qs):
    Zillow[col] = Zillow[q].mean(axis=1)
        
Zillow = Zillow[column_names]

In [82]:
Zillow

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [83]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    Zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
    Zillow.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    Zillow['State'] = Zillow['State'].map(states)
    Zillow.set_index(['State','RegionName'],inplace=True)
    col = list(Zillow.columns)
    col = col[0:45]
    Zillow.drop(col,axis=1,inplace=1)
    
    Zillow_column_names = Zillow.columns.tolist()
    Zillow_column_month = [x for x in Zillow_column_names if x[0].isdigit()]
    Quarter_List = []
    One_Quarter = []
    Quarter_dict = {}
    for month in Zillow_column_month:
        currnt_quarter = month_to_quarter(month)
        if(currnt_quarter not in Quarter_List):
            Quarter_List.append(currnt_quarter) 
            if(len(One_Quarter)!=0):
                Quarter_dict[currnt_quarter] =One_Quarter
            One_Quarter = []
            One_Quarter.append(month)
        else:
            One_Quarter.append(month)
            Quarter_dict[currnt_quarter] =One_Quarter
            
    for k, v in Quarter_dict.items():
        Zillow[k] = Zillow[v].mean(axis=1)
    Quarter_Names = list(Quarter_dict.keys())
    Quarter_Names.sort()
    Zillow = Zillow[Quarter_Names]
    
    return Zillow

In [84]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [85]:
def convert_housing_data_to_quarters_Approach2():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    data['State'] = data['State'].map(states)
    data.set_index(['State','RegionName'],inplace=True)
    col = list(data.columns)
    col = col[0:45]
    data.drop(col,axis=1,inplace=1)

    #qs is the quarters of the year
    qs = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # new columns
    column_names = new_col_names()
    for col,q in zip(column_names,qs):
        data[col] = data[q].mean(axis=1)
        
    data = data[column_names]
    
    return data

In [86]:
convert_housing_data_to_quarters_Approach2()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


## Problem 6

In [91]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    unitowns = get_list_of_university_towns()
    Zillow = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_start = Zillow.columns[Zillow.columns.get_loc(recession_start) -1]
    recession_bottom = get_recession_bottom()
    Zillow_in_recession = Zillow[[recession_bottom, recession_start]]
    Zillow_in_recession['ratio']=Zillow_in_recession[recession_bottom]-Zillow_in_recession[recession_start]
    Zillow_in_recession = Zillow_in_recession.reset_index()
    
    unitowns_house = pd.merge(Zillow_in_recession,unitowns,how='inner',on=['State','RegionName'])
    unitowns_house['uni'] = True
    unitowns_house = pd.merge(Zillow_in_recession,unitowns_house,how='outer',on=['State','RegionName',recession_bottom,recession_start,'ratio'])
    unitowns_house['uni'] = unitowns_house['uni'].fillna(False)
    uni_house = unitowns_house[unitowns_house['uni'] == True]
    not_uni_house = unitowns_house[unitowns_house['uni'] == False]
    t,p = ttest_ind(uni_house['ratio'].dropna(),not_uni_house['ratio'].dropna())

    different = True if p < 0.01 else False

    better = "non-university town" if uni_house['ratio'].mean() < not_uni_house['ratio'].mean() else "university town"
    
    return different, p, better

In [92]:
run_ttest()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.002099659657952052, 'university town')